In [1]:
import os
import pandas as pd
import arcpy
import datetime

arcpy.env.overwriteOutput = True

Script to process earth quake data for a 3d point density demonstration.


In [2]:
# Set up path variables

base_dir = r'C:\test\blog\3dPointDensity\eq'  # Base directory I am working in, change to match your folder structure
os.chdir(base_dir)
gdb_name = 'Earthquake_density.gdb'
base_gdb = os.path.join(base_dir, gdb_name)

# If it exists, delete the old working geodatabase for the script
if arcpy.Exists(base_gdb):
    arcpy.Delete_management(base_gdb)

# Create a geodatabase for the script to work in
if not arcpy.Exists(base_gdb):
    arcpy.management.CreateFileGDB(base_dir, gdb_name, "CURRENT")
    
    

# Data Source

Earthquake data downloaded from the USGS https://earthquake.usgs.gov/fdsnws/event/1/ The data covers the time period from 2007-2017 and is from 33 to 37 degrees longitude and -103 to -94 degrees longitude. The data is shared as a feature service at https://arcg.is/Obv99 or a shapefile at https://arcg.is/01HqX40

After downloading the shapefile extract it to your base directory.

# Spatial Data Analysis

Starting with the Oklahoma_EQ shapefile downloaded to your base directory.

In [ ]:
input_fc_fullpath = os.path.join(base_dir, "Oklahoma_EQ.shp")



### Spatial Weights Matrix

Now the input data is a feature class with x, y and z coordinates. First create a spatial weights matrix. Using a fixed distance band the tool will count all of the spatial neighbours within the threshold. Since the distance is fixed it will be possible to compare how many neighbours each point has.

In [ ]:
# Create spatial weights matrix
swm_name = "Oklahoma_10000m.swm"
swm_path = os.path.join(base_dir, swm_name)
if os.path.exists(swm_path):  # Check if the swm file exists, if so delete it
    os.remove(swm_path)
r1 = arcpy.stats.GenerateSpatialWeightsMatrix(input_fc_fullpath, "MYID", 
                                         swm_path, "FIXED_DISTANCE", "EUCLIDEAN", 1, 10000, 0, 
                                         "NO_STANDARDIZATION", None, None, None, None, "USE_Z_VALUES")
# Convert SWM to table
swm_table = "Oklahoma_10000m"
swm_table_fullpath = os.path.join(base_gdb, swm_table)
arcpy.stats.ConvertSpatialWeightsMatrixtoTable(swm_path, swm_table_fullpath)

# Calculate summary statistics on the SWM table to get the numebr of neighbours within 10000 m for each point
swm_summary_table = "Oklahoma_10000_Summary"
swm_summary_table_fullpath = os.path.join(base_gdb, swm_summary_table)
arcpy.analysis.Statistics(swm_table_fullpath, swm_summary_table_fullpath, "NID COUNT", "MYID")


#Make a feature layer and a table view for the points and swm table
eq_layer_name = "Oklahoma_EQ_Neighbours"
arcpy.MakeFeatureLayer_management(input_fc_fullpath, eq_layer_name)
table_view_name = "Oklahoma_neighbour_swm"
arcpy.MakeTableView_management(swm_summary_table_fullpath, table_view_name)

# Add join to the original data and the table
arcpy.management.AddJoin(eq_layer_name, "MYID", table_view_name, "MYID", "KEEP_ALL")
# arcpy.management.AddJoin(r"C:\test\blog\3dPointDensity\3dPointDensity.gdb\Oklahoma_eq", "MyID", r"C:\test\blog\3dPointDensity\3dPointDensity.gdb\Oklahoma_eq_swm", "MyID", "KEEP_ALL")

# Copy the joined table to a new file
eq_joined_pre_agg = "Oakalhoma_neighbour"
eq_joined_pre_agg_fullpath = os.path.join(base_gdb, eq_joined_pre_agg)
arcpy.management.CopyFeatures(eq_layer_name, eq_joined_pre_agg_fullpath, None, None, None, None)
print(eq_joined_pre_agg_fullpath)

In [ ]:
# Check the detailed result messages
for i in range(r1.messageCount):
    print(r1.getMessage(i))